This notebook is for saving 5-yr averages of SST, SSS and SSH from 1/4 and 1/8 simulations.

In [1]:
import xarray as xr
import numpy as np
from xgcm import Grid
import filter
from dask.diagnostics import ProgressBar
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
import glob, os

from dask.distributed import Client
from dask.distributed import LocalCluster
cluster = LocalCluster()
client = Client(cluster)

client

Client Scheduler: tcp://127.0.0.1:35698 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 16 Memory: 406.01 GB


In [2]:
# This is for 1/8 deg run

path_ssh = "/archive/Raphael.Dussin/FMS2019.01.03_devgfdl_20210308/CM4_piControl_c192_OM4p125_v3/" + \
"gfdl.ncrc4-intel18-prod-openmp/pp/ocean_daily/ts/daily/5yr/"

filelist_ssh = glob.glob(path_ssh + "*.zos.nc")
filelist_ssh.sort()
filelist_sst = glob.glob(path_ssh + "*.tos.nc")
filelist_sst.sort()
filelist_sss = glob.glob(path_ssh + "*.sos.nc")
filelist_sss.sort()

path_grid = "/archive/Raphael.Dussin/FMS2019.01.03_devgfdl_20210308/CM4_piControl_c192_OM4p125_v3/" + \
"gfdl.ncrc4-intel18-prod-openmp/pp/ocean_monthly/"
ds_grid = xr.open_dataset(path_grid + "ocean_monthly.static.nc")
ds_grid1 = ds_grid.isel(xq = slice(0,2880), yq=slice(0,2240))

save_path = "/archive/Hemant.Khatri/MOM_Budget/"
ds_full = []
# Loop for reading data and saving 5-yr averages (100 - 145 years)
for i in range(19, 28):
    
    tim_yr = filelist_ssh[i][-24:-7]
    print("Year Running = ", tim_yr)
    
    ds_ssh = xr.open_dataset(filelist_ssh[i], decode_times=False)
    ds_sst = xr.open_dataset(filelist_sst[i], decode_times=False)
    ds_sss = xr.open_dataset(filelist_sss[i+1], decode_times=False)
    
    %time ds = xr.merge([ds_ssh, ds_sst, ds_sss]).mean('time')
    
    ds_full.append(ds)
    
ds1 = xr.concat(ds_full, dim='time')

ds_save = xr.merge([ds1, ds_grid1])

times = np.linspace(102.5, 142.5, 9)
ds_save.coords['time'] = times
ds_save.time.attrs['units'] = "years since 0001"
    
save_file = save_path + "OM4p125/OM4p125_TS.nc"

ds_save = ds_save.load()
ds_save.to_netcdf(save_file)

print("File Saved Successfully")

Year Running =  02010101-02051231
Year Running =  02060101-02101231
Year Running =  02110101-02151231
Year Running =  02160101-02201231
Year Running =  02210101-02251231
Year Running =  02260101-02301231
Year Running =  02310101-02351231
Year Running =  02360101-02401231
Year Running =  02410101-02451231
File Saved Successfully


In [11]:
# This is for 1/4 deg run

path_ssh = "/archive/Raphael.Dussin/FMS2019.01.03_devgfdl_20210308/CM4_piControl_c192_OM4p25/" + \
"gfdl.ncrc4-intel18-prod-openmp/pp/ocean_daily/ts/daily/5yr/"

filelist_ssh = glob.glob(path_ssh + "*.zos.nc")
filelist_ssh.sort()
filelist_sst = glob.glob(path_ssh + "*.tos.nc")
filelist_sst.sort()
filelist_sss = glob.glob(path_ssh + "*.sos.nc")
filelist_sss.sort()

path_grid = "/archive/Raphael.Dussin/FMS2019.01.03_devgfdl_20210308/CM4_piControl_c192_OM4p25/" + \
"gfdl.ncrc4-intel18-prod-openmp/pp/ocean_monthly/"
ds_grid = xr.open_dataset(path_grid + "ocean_monthly.static.nc")
ds_grid1 = ds_grid.isel(xq = slice(1,1441), yq=slice(1,1081))

save_path = "/archive/Hemant.Khatri/MOM_Budget/"
ds_full = []
# Loop for reading data and saving 5-yr averages (100 - 145 years)
for i in range(19, 28):
    
    tim_yr = filelist_ssh[i][-24:-7]
    print("Year Running = ", tim_yr)
    
    ds_ssh = xr.open_dataset(filelist_ssh[i], decode_times=False)
    ds_sst = xr.open_dataset(filelist_sst[i], decode_times=False)
    ds_sss = xr.open_dataset(filelist_sss[i], decode_times=False)
    
    %time ds = xr.merge([ds_ssh, ds_sst, ds_sss]).mean('time') #chunk({'xh':360, 'yh':360})
    
    ds_full.append(ds)

ds1 = xr.concat(ds_full, dim='time')

ds_save = xr.merge([ds1, ds_grid1])

times = np.linspace(102.5, 142.5, 9)
ds_save.coords['time'] = times
ds_save.time.attrs['units'] = "years since 0001"
    
save_file = save_path + "OM4p25/OM4p25_TS.nc"

ds_save = ds_save.load()
ds_save.to_netcdf(save_file)

print("File Saved Successfully")

Year Running =  01010101-01051231
Year Running =  01060101-01101231
Year Running =  01110101-01151231
Year Running =  01160101-01201231
Year Running =  01210101-01251231
Year Running =  01260101-01301231
Year Running =  01310101-01351231
Year Running =  01360101-01401231
Year Running =  01410101-01451231


AttributeError: 'list' object has no attribute 'concat'

#### Some checks

In [37]:
path_ssh = "/archive/Raphael.Dussin/FMS2019.01.03_devgfdl_20210308/CM4_piControl_c192_OM4p125_v3/" + \
"gfdl.ncrc4-intel18-prod-openmp/pp/ocean_daily/ts/daily/5yr/"

filelist_ssh = glob.glob(path_ssh + "*.zos.nc")
filelist_ssh.sort()
filelist_sst = glob.glob(path_ssh + "*.tos.nc")
filelist_sst.sort()
filelist_sss = glob.glob(path_ssh + "*.sos.nc")
filelist_sss.sort()

In [42]:
filelist_sss[20][-24:-7]

'02010101-02051231'